In [28]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets,
    decode_clean_actors,
    decode_clean,

)
import eda_movies as eda

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
link = "big_dataframe/actors.parquet"
test = pd.read_parquet(link)
test

,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,nm0000001,Fred Astaire,1899,"[""Guy Holden""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
1,nm0000001,Fred Astaire,1899,"[""Huck Haines""]",2,"tt0050419,tt0072308,tt0053137,tt0031983"
2,nm0000001,Fred Astaire,1899,"[""Jerry Travers""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
3,nm0000001,Fred Astaire,1899,"[""Bake Baker""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
4,nm0000001,Fred Astaire,1899,"[""Lucky Garnett""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
...,...,...,...,...,...,...
1183372,nm9993616,Ryan Mac Lennan,\N,"[""Mick Owen""]",4,tt4844148
1183373,nm9993650,Marcin Balcerak,\N,"[""Guardian""]",4,tt8739208
1183374,nm9993693,Apsara Rani,1996,\N,4,"tt12856788,tt8737752,tt8302382,tt13847502"
1183375,nm9993693,Apsara Rani,1996,"[""Vidhi""]",2,"tt12856788,tt8737752,tt8302382,tt13847502"


In [30]:
link = "big_dataframe/actors.parquet"
df_og = import_datasets(
    link,
    types="parquet",
)
cleaning = DataCleaner()
df = cleaning.fix_values(df_og, "fix_n")
# df = cleaning.fix_values(df1, "fix_encode")

2023-10-26 16:51:09 INFO     Parquet loaded ! Importing actors...
2023-10-26 16:51:10 INFO     Fixing N values...


In [31]:
df

,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,nm0000001,Fred Astaire,1899,"[""Guy Holden""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
1,nm0000001,Fred Astaire,1899,"[""Huck Haines""]",2,"tt0050419,tt0072308,tt0053137,tt0031983"
2,nm0000001,Fred Astaire,1899,"[""Jerry Travers""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
3,nm0000001,Fred Astaire,1899,"[""Bake Baker""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
4,nm0000001,Fred Astaire,1899,"[""Lucky Garnett""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
...,...,...,...,...,...,...
1183372,nm9993616,Ryan Mac Lennan,0,"[""Mick Owen""]",4,tt4844148
1183373,nm9993650,Marcin Balcerak,0,"[""Guardian""]",4,tt8739208
1183374,nm9993693,Apsara Rani,1996,0,4,"tt12856788,tt8737752,tt8302382,tt13847502"
1183375,nm9993693,Apsara Rani,1996,"[""Vidhi""]",2,"tt12856788,tt8737752,tt8302382,tt13847502"


In [32]:
df["person_birthdate"] = df["person_birthdate"].astype("int64")

In [33]:
df.isna().sum()

person_id           0
person_name         0
person_birthdate    0
person_role         0
person_index        0
person_film         0
dtype: int64

In [34]:
df.dtypes

person_id           object
person_name         object
person_birthdate     int64
person_role         object
person_index         int64
person_film         object
dtype: object

In [35]:
df["person_film"] = df["person_film"].str.split(",")

In [36]:
df["person_role"] = np.where(df["person_role"] == 0, "Unknown", df["person_role"])

In [37]:
df.isna().sum()

person_id             0
person_name           0
person_birthdate      0
person_role           0
person_index          0
person_film         739
dtype: int64

In [38]:
df["person_film"] = np.where(df["person_film"].isna(), "Unknown", df["person_film"])

In [39]:
mf = df.copy()

In [40]:
mf["person_role"] = mf["person_role"].apply(decode_clean_actors).str.split(",")

In [41]:
mf.head()

,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,nm0000001,Fred Astaire,1899,[Guy Holden],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"
1,nm0000001,Fred Astaire,1899,[Huck Haines],2,"[tt0050419, tt0072308, tt0053137, tt0031983]"
2,nm0000001,Fred Astaire,1899,[Jerry Travers],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"
3,nm0000001,Fred Astaire,1899,[Bake Baker],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"
4,nm0000001,Fred Astaire,1899,[Lucky Garnett],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"


In [42]:
mf[mf["person_name"].str.contains('Eddie Murphy')]

,person_id,person_name,person_birthdate,person_role,person_index,person_film
18952,nm0000552,Eddie Murphy,1961,[Reggie Hammond],2,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18953,nm0000552,Eddie Murphy,1961,[Billy Ray Valentine],1,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18954,nm0000552,Eddie Murphy,1961,[Lieutenant T.M. Landry],2,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18955,nm0000552,Eddie Murphy,1961,[Axel Foley],1,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18956,nm0000552,Eddie Murphy,1961,[Chandler Jarrell],1,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18957,nm0000552,Eddie Murphy,1961,[Axel Foley],1,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18958,nm0000552,Eddie Murphy,1961,[Self],1,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18959,nm0000552,Eddie Murphy,1961,"[Prince Akeem, Clarence, Saul]",1,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18960,nm0000552,Eddie Murphy,1961,[Quick],1,"[tt0097481, tt0117218, tt0094898, tt0443489]"
18961,nm0000552,Eddie Murphy,1961,[Reggie Hammond],1,"[tt0097481, tt0117218, tt0094898, tt0443489]"


In [43]:
mf.shape

(1183377, 6)

In [44]:
# merge avec movies

In [45]:
link = "clean_datasets/movies_cleaned.parquet"
link_principals = "movies_datasets/title_principals.tsv"

movies_cleaned = import_datasets(link, "parquet")
principals = import_datasets(link_principals, "pandas", sep="\t")

2023-10-26 16:51:20 INFO     Parquet loaded ! Importing movies_cleaned...
2023-10-26 16:51:20 INFO     Pandas loaded ! Importing title_principals...


In [46]:
principals = principals[["tconst", "nconst"]]

principals.head()

,tconst,nconst
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0374658
3,tt0000002,nm0721526
4,tt0000002,nm1335271


In [47]:
movies_with_personid = pd.merge(
    movies_cleaned,
    principals,
    left_on = "titre_id",
    right_on = "tconst"
)

In [48]:
movies_with_personid = movies_with_personid.drop("tconst", axis=1)

movies_with_personid.head()

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,titleId,region,cuts,nconst
0,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,nm0941700
1,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,nm0409869
2,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,nm0562920
3,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,nm0103722
4,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,nm0751245


In [49]:
movies_with_personid.shape

(352804, 18)

In [50]:
actors_merged = pd.merge(
    movies_with_personid,
    mf,
    left_on = "nconst",
    right_on = "person_id"
)

actors_merged = actors_merged.drop("nconst", axis=1)

In [51]:
actors_merged

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,...,status,titleId,region,cuts,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Pfc. Windy Craven],4,"[tt0040724, tt0054331, tt0039748, tt0041113]"
1,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Duke Martin],1,"[tt0040724, tt0054331, tt0039748, tt0041113]"
2,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Paul Lester],1,"[tt0040724, tt0054331, tt0039748, tt0041113]"
3,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Fantail],4,"[tt0040724, tt0054331, tt0039748, tt0041113]"
4,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Jack Burden],2,"[tt0040724, tt0054331, tt0039748, tt0041113]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116184,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,Released,tt9916362,FR,>2020,nm3766704,Amaia Aberasturi,1997,[Leire],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"
3116185,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,Released,tt9916362,FR,>2020,nm3766704,Amaia Aberasturi,1997,[Unknown],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"
3116186,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,Released,tt9916362,FR,>2020,nm3766704,Amaia Aberasturi,1997,[Begoña],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"
3116187,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,Released,tt9916362,FR,>2020,nm3766704,Amaia Aberasturi,1997,[Ana],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"


In [61]:
len(actors_merged[actors_merged["person_name"].str.contains("Gérard Depardieu")])

18724

In [55]:
actors_merged.isna().sum()

titre_id                0
titre_str               0
titre_date_sortie       0
titre_duree             0
titre_genres            0
rating_avg              0
rating_votes            0
original_language       0
original_title          0
popularity              0
production_countries    0
revenue                 0
spoken_languages        0
status                  0
titleId                 0
region                  0
cuts                    0
person_id               0
person_name             0
person_birthdate        0
person_role             0
person_index            0
person_film             0
dtype: int64